In [1]:
# basic imports
import numpy as np

# causallearn imports
from causallearn.search.ConstraintBased.FCI import fci

# utils imports
from utils import basic_causal_dataframe

/opt/anaconda3/envs/tesi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(0)

In [3]:
data = basic_causal_dataframe()


In [4]:
# extract the variable names
variable_names = list(data.columns)

# Pass the variable names directly to the FCI algorithm
g, edges = fci(data.to_numpy(), node_names=variable_names)

Depth=0, working on node 2:  67%|██████▋   | 2/3 [00:00<00:00, 174.06it/s]

Depth=0, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 197.28it/s]


In [5]:
# Verify node names
[node.get_name() for node in g.nodes]

['X', 'Y', 'Z']

In [6]:
from causallearn.utils.GraphUtils import GraphUtils

pdy = GraphUtils.to_pydot(g)
pdy.write_png('simple_test.png')

In [7]:
g.graph

array([[0, 2, 2],
       [2, 0, 2],
       [2, 2, 0]])

In [9]:
edges = []
indices = np.where(g.graph != 0)
processed_pairs = set()

In [10]:
for i, j in zip(indices[0], indices[1]):
    node_pair = frozenset([i.item(), j.item()])
    
    if node_pair in processed_pairs:
        continue
    
    if g.graph[i, j] == -1 and g.graph[j, i] == 1:
        edges.append({
            'from': g.nodes[i].get_name(),
            'to': g.nodes[j].get_name(),
            'type': "->"
        })
    elif g.graph[i, j] == 2 and g.graph[j, i] == 1:
        edges.append({
            'from': g.nodes[j].get_name(),
            'to': g.nodes[i].get_name(),
            'type': "o->"
        })
    elif g.graph[i, j] == 2 and g.graph[j, i] == 2:
        edges.append({
            'from': g.nodes[i].get_name(),
            'to': g.nodes[j].get_name(),
            'type': "o-o"
        })
        processed_pairs.add(node_pair)
    elif g.graph[i, j] == 1 and g.graph[j, i] == 1:
        edges.append({
            'from': g.nodes[i].get_name(),
            'to': g.nodes[j].get_name(),
            'type': "<->"
        })
        processed_pairs.add(node_pair)

In [11]:
edges

[{'from': 'X', 'to': 'Y', 'type': 'o-o'},
 {'from': 'X', 'to': 'Z', 'type': 'o-o'},
 {'from': 'Y', 'to': 'Z', 'type': 'o-o'}]

In [12]:
# create a list of possible causal relationship in the variables
causal_dict = {}
for node1 in variable_names:
    for node2 in variable_names:
        for edge in ['direct', 'latent']:
            causal_dict[(node1, node2, edge)] = len(causal_dict) + 1

In [14]:
reversed_causal_dict = {v: k for k, v in causal_dict.items()}
reversed_causal_dict

{1: ('X', 'X', 'direct'),
 2: ('X', 'X', 'latent'),
 3: ('X', 'Y', 'direct'),
 4: ('X', 'Y', 'latent'),
 5: ('X', 'Z', 'direct'),
 6: ('X', 'Z', 'latent'),
 7: ('Y', 'X', 'direct'),
 8: ('Y', 'X', 'latent'),
 9: ('Y', 'Y', 'direct'),
 10: ('Y', 'Y', 'latent'),
 11: ('Y', 'Z', 'direct'),
 12: ('Y', 'Z', 'latent'),
 13: ('Z', 'X', 'direct'),
 14: ('Z', 'X', 'latent'),
 15: ('Z', 'Y', 'direct'),
 16: ('Z', 'Y', 'latent'),
 17: ('Z', 'Z', 'direct'),
 18: ('Z', 'Z', 'latent')}